# Hands on session: Analysis of Higgs data

In [6]:
%reload_ext custom_magics

In [7]:
num_workers=5
spark=%sc $num_workers
#check if spark is there
spark

<SparkContext master=k8s://https://192.168.2.39:6443 appName=jupyter-leggerf>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Check out these custom functions
from custom_functions import *

In [8]:
from pyspark.sql import SparkSession
spark_session = SparkSession(spark)

## Prepare the input data

In [9]:
#read input dataset from parquet

inputFile = 'hdfs://192.168.2.39/data/Higgs10M.parquet' 
#inputFile = 'hdfs://192.168.2.39/data/Higgs100k.parquet' 
#other files are: Higgs1M.parquet, Higgs100k.parquet

%time df = spark_session.read.format('parquet').option('header', 'true').option('inferschema', 'true').load(inputFile)

CPU times: user 14.1 ms, sys: 5.14 ms, total: 19.3 ms
Wall time: 5.21 s


In [6]:
total_events = df.count()

print('There are '+str(total_events)+' events')

There are 10000000 events


#### QUESTION 1: is the dataset unbalanced? Do we need undersampling?

In [1]:
# motivate your answer here


#### QUESTION 2: Split the dataset into 5 parts with random events

In [2]:
#your answer goes here


#### QUESTION 3: split the dataset for training and test

In [4]:
# your answer goes here
# split into two dataframes called training and test

#### Create the input feature vector

Documentation on how to do data preprocessing in Spark: https://spark.apache.org/docs/latest/ml-features.html

In [10]:
feature = train.columns
feature.remove('label')

# VectorAssembler is a transformer that combines a given list of columns into a single vector column
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=feature, outputCol='features')

train = assembler.transform(train)
test = assembler.transform(test)

## ML with pyspark

Try to solve the signal vs background classification problem using a new algorithm of your choice (obviously, not GBT or MCP)
Documentation at: https://spark.apache.org/docs/2.2.0/ml-classification-regression.html

In [11]:
from pyspark.ml.classification import *

Let's fit the model on the **training dataset**:

In [1]:
# your answer goes here

#### QUESTION 4: Compute the accuracy on the test dataset

In [5]:
#your answer goes here

#### QUESTION 5: Are we over/under-fitting the data? How could the accuracy be improved?

In [2]:
#your answer goes here

#### Hyperparameter tuning

We use the *spark_sklearn.grid_search* method.

In [18]:
subSample = train.sample(fraction=1.0/10, withReplacement=False).cache()
print('We will use',subSample.count(),' events for the grid search.')

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC')

We will use 801432  events for the grid search.


##### QUESTION 6: Compare n k-cross validator vs TrainValidation split methods 

In [6]:
#your answer goes here


#### QUESTION 7: do you get a better accuracy? Why? Are you over/underfitting?

#### QUESTION 8

pick any two models (can be any from pyspark MLLib, Keras Sequential, BigDL sequential) and:
- compare them in terms of performances (you can use results from previous days)
- compare how they scale in terms of performances and training time when you change the number of executors (for example, from 1 to 5)
- discuss the results

In [5]:
# When you're done, stop spark
spark.stop()